<a href="https://colab.research.google.com/github/AndreColombo/Verificador-Fake-News/blob/main/Desafio_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-cloud-language
!pip install -q nltk

In [ ]:
import nltk
nltk.download('vader_lexicon')
from google.cloud import language_v1
from google.oauth2 import service_account
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Função para analisar o sentimento do texto

In [ ]:
def analisar_sentimento(texto):
    sia = SentimentIntensityAnalyzer()
    sentimento = sia.polarity_scores(texto)
    return sentimento

Função para analisar entidades nomeadas com a API do Google

In [ ]:
def analisar_entidades(texto):
    api_key = input("Digite sua API Key do Google Cloud: ")
    credentials = service_account.Credentials.from_api_key(api_key)
    client = language_v1.LanguageServiceClient(credentials=credentials)
    document = language_v1.Document(content=texto, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_entities(document=document)
    return response

Função para verificar a confiabilidade da fonte

In [ ]:
def verificar_fonte(fonte):
    fontes_confiaveis = ["https://www.cnnbrasil.com.br/", "https://www.bbc.com/"]
    if fonte in fontes_confiaveis:
        return True
    else:
        return False

Função principal para analisar a notícia

In [ ]:
def analisar_noticia(texto, fonte=None):
    # Análise de Sentimento
    sentimento = analisar_sentimento(texto)

    # Análise de Entidades
    entidades = analisar_entidades(texto)

    # Verificação da Fonte
    confiabilidade_fonte = verificar_fonte(fonte) if fonte else None

    print("Análise da Notícia:")
    print("Sentimento:", sentimento)
    print("\nEntidades:")
    for entidade in entidades.entities:
        print(f"Nome: {entidade.name}, Tipo: {entidade.type_}")

    print("\nConfiabilidade da Fonte:", confiabilidade_fonte)

    # Calcular um índice de confiabilidade
    indice = 0
    if sentimento['neg'] > 0.5: indice -= 0.3
    if confiabilidade_fonte == False: indice -= 0.5
    print("Índice de Confiabilidade: ", indice)

Exemplo de uso

In [ ]:
texto_noticia = """
EUA dizem ter derrubado um objeto não identificado sobre o Alasca
Os Estados Unidos derrubaram um "objeto de alta altitude" não identificado sobre o Alasca, disse o governo americano nesta sexta-feira (10).
Um porta-voz do Conselho de Segurança Nacional, John Kirby, disse a repórteres que o objeto, que ele disse ser do tamanho de um "carro pequeno", representava "uma ameaça razoável ao tráfego aéreo civil".
O objeto estava voando a uma altitude de cerca de 40.000 pés (12.000 metros) e foi derrubado por volta das 13h45, horário do leste (15h45 no horário de Brasília), acrescentou.
O presidente dos Estados Unidos, Joe Biden, ordenou a derrubada, de acordo com Kirby.
Ele disse que o objeto caiu em águas congeladas dos EUA, perto da fronteira canadense.
Não está claro quem é o dono do objeto ou de onde ele veio, de acordo com Kirby.
"""

fonte_noticia = "https://www.bbc.com/news/articles/c4n14merj97o"

analisar_noticia(texto_noticia, fonte_noticia)